In [ ]:
import math
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
nltk.download('punkt')

In [ ]:
data_dir = "data"
file_path = data_dir + "/en_US.twitter.txt"

## nltk settings
nltk.data.path.append(data_dir)
nltk.download('punkt')

## Opening the File in read mode ("r")
with open(file_path, "r") as f:
    data = f.read()

In [ ]:
def preprocess_pipeline(data):
    # Split by newline character, strip spaces, drop empty lines, convert to lowercase, and tokenize
    return [nltk.word_tokenize(sentence.lower().strip()) for sentence in data.split('\n') if sentence.strip()]

In [ ]:
## Pass our data to this function    
tokenized_sentences = preprocess_pipeline(data)
print(tokenized_sentences)

In [ ]:
## Obtain Train and Test Split 
train, test = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)

## Obtain Train and Validation Split 
train, val = train_test_split(train, test_size=0.25, random_state=42)

In [ ]:
# Frequency Dictionary
def count_the_words(sentences) -> 'dict':
    # Creating a Dictionary of counts
    word_counts = {}
    for sentence in sentences:
        for token in sentence:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

In [ ]:
# Handling Out-of-vocabulary words
def handling_oov(tokenized_sentences, count_threshold) -> 'list':
    # Empty list for closed vocabulary
    closed_vocabulary = []
    word_cound = count_the_words(tokenized_sentences)
    for word, count in words_count.items():
        if count >= count_threshold :
            closed_vocabulary.append(word)
    return closed_vocabulary

In [ ]:
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>") -> 'list':

  # Convert Vocabulary into a set
  vocabulary = set(vocabulary)

  # Create empty list for sentences
  new_tokenized_sentences = []
  
  # Iterate over sentences
  for sentence in tokenized_sentences:

    # Iterate over sentence and add <unk> 
    # if the token is absent from the vocabulary
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)
    
    # Append sentece to the new list
    new_tokenized_sentences.append(new_sentence)

  return new_tokenized_sentences

In [ ]:
def cleansing(train_data, test_data, count_threshold):
    
  # Get closed Vocabulary
  vocabulary = handling_oov(train_data, count_threshold)
    
  # Updated Training Dataset
  new_train_data = unk_tokenize(train_data, vocabulary)
    
  # Updated Test Dataset
  new_test_data = unk_tokenize(test_data, vocabulary)

  return new_train_data, new_test_data, vocabulary

In [ ]:
min_freq = 6
final_train, final_test, vocabulary = cleansing(train, test, min_freq)

In [ ]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>") -> 'dict':

  # Empty dict for n-grams
  n_grams = {}
 
  # Iterate over all sentences in the dataset
  for sentence in data:
        
    # Append n start tokens and a single end token to the sentence
    sentence = [start_token]*n + sentence + [end_token]
    
    # Convert the sentence into a tuple
    sentence = tuple(sentence)

    # Temp var to store length from start of n-gram to end
    m = len(sentence) if n==1 else len(sentence)-1
    
    # Iterate over this length
    for i in range(m):
        
      # Get the n-gram
      n_gram = sentence[i:i+n]
    
      # Add the count of n-gram as value to our dictionary
      # IF n-gram is already present
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      # Add n-gram count
      else:
        n_grams[n_gram] = 1
        
  return n_grams

In [ ]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k = 1.0) -> 'float':

  # Convert the previous_n_gram into a tuple 
  previous_n_gram = tuple(previous_n_gram)
    
  # Calculating the count, if exists from our freq dictionary otherwise zero
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  
  # The Denominator
  denom = previous_n_gram_count + k * vocabulary_size

  # previous n-gram plus the current word as a tuple
  nplus1_gram = previous_n_gram + (word,)

  # Calculating the nplus1 count, if exists from our freq dictionary otherwise zero 
  nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  # Numerator
  num = nplus1_gram_count + k

  # Final Fraction
  prob = num / denom
  return prob

In [ ]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0) -> 'dict':

  # Convert to Tuple
  previous_n_gram = tuple(previous_n_gram)

  # Add end and unknown tokens to the vocabulary
  vocabulary = vocabulary + ["<e>", "<unk>"]

  # Calculate the size of the vocabulary
  vocabulary_size = len(vocabulary)

  # Empty dict for probabilites
  probabilities = {}

  # Iterate over words 
  for word in vocabulary:
    
    # Calculate probability
    probability = prob_for_single_word(word, previous_n_gram, 
                                           n_gram_counts, nplus1_gram_counts, 
                                           vocabulary_size, k=k)
    # Create mapping: word -> probability
    probabilities[word] = probability

  return probabilities


In [ ]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0, start_with=None):

    
    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 
    
    # most recent 'n' words
    previous_n_gram = previous_tokens[-n:]
    
    # Calculate probabilty for all words
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    # Intialize the suggestion and max probability
    suggestion = None
    max_prob = 0

    # Iterate over all words and probabilites, returning the max.
    # We also add a check if the start_with parameter is provided
    for word, prob in probabilities.items():
        
        if start_with != None: 
            
            if not word.startswith(start_with):
                continue 

        if prob > max_prob: 

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):

    # See how many models we have
    count = len(n_gram_counts_list)
    
    # Empty list for suggestions
    suggestions = []
    
    # IMP: Earlier "-1"
    
    # Loop over counts
    for i in range(count-1):
        
        # get n and nplus1 counts
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]
        
        # get suggestions 
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        # Append to list
        suggestions.append(suggestion)
        
    return suggestions

In [ ]:
n_gram_counts_list = []
for n in range(1, 6):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

In [ ]:
previous_tokens = ["i", "was", "about"]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

display(suggestion)

In [ ]:
print("unigram count:" , len(n_gram_counts_list[0]))
print("bigram count:", len(n_gram_counts_list[1]))
print("trigram count:", len(n_gram_counts_list[2]))
print("quadgram count:", len(n_gram_counts_list[3]))
print("quintgram count:", len(n_gram_counts_list[4]))

In [ ]:
# Storing to file
with open("en_counts.txt", 'wb') as f:
    pickle.dump(n_gram_counts_list, f)

In [ ]:
# Storing to file
with open("vocab.txt", 'wb') as f:
    pickle.dump(vocabulary, f)